## Extract TRILEGAL simulated catalog for a field defined by RA and Dec, and analyze results

In [1]:
from dl import authClient as ac, queryClient as qc
from dl.helpers.utils import convert
from getpass import getpass

In [167]:
## automatically reload any modules read below that might have changed (e.g. plots)
%reload_ext autoreload
%autoreload 2
# importing plotting and locus tools: 
import LocusTools as lt
import BayesTools as bt
import PlotTools as pt
import MakePriors as mp

In [3]:
%matplotlib inline
import sys
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import hstack
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
from scipy import optimize
from scipy import interpolate 
from scipy.interpolate import griddata
from scipy.stats import gaussian_kde

## dump maps with priors and simulated catalog data for BDBS field #1

In [168]:
# this is for BDBS patch #1 (BDBS data file: field_044.fit)
queryBDBSpatch1 = mp.makeQueryString(269.0, 271.0, -36.0, -34.0)
print(queryBDBSpatch1)
BDBSpatch1 = mp.retrievePatch(queryBDBSpatch1)
print('retrieved', len(BDBSpatch1), 'entries')


    SELECT ra, dec, gall, galb, gc, logage, mass, label, logg, m_h, av, mu0, umag, gmag, rmag, imag, zmag, ymag, nest4096
    FROM lsst_sim.simdr2
    WHERE 269.0 < ra and ra < 271.0 AND -36.0 < dec and dec < -34.0 AND umag < 99.99 

querying...
converting...
retrieved patch with 23448617  entries
retrieved 23448617 entries


In [170]:
BDBSpatch1

,ra,dec,gall,galb,gc,logage,mass,label,logg,FeH,...,u,g,r,i,z,ug,gr,ri,iz,gi
0,269.007075,-35.998518,355.035081,-5.487037,3,10.11,0.639,1,4.635,-0.74,...,23.155812,21.678661,21.182758,21.018291,20.969389,1.477151,0.495903,0.164467,0.048902,0.660370
1,269.003933,-35.999876,355.032629,-5.485505,4,10.01,0.839,1,4.494,0.02,...,21.760881,20.020919,19.537371,19.432001,19.428609,1.739962,0.483548,0.105370,0.003391,0.588919
2,269.004751,-35.999824,355.033005,-5.486054,1,9.73,0.292,1,4.942,0.11,...,29.896433,27.075274,25.856339,24.363355,23.702132,2.821159,1.218935,1.492984,0.661223,2.711919
3,269.004547,-35.999735,355.033000,-5.485866,1,8.07,0.477,0,4.731,0.11,...,26.375094,23.705758,22.610113,21.497957,21.018272,2.669336,1.095645,1.112156,0.479684,2.207801
4,269.004717,-35.999753,355.033053,-5.485994,4,10.01,0.696,1,4.609,0.17,...,26.033883,23.214274,22.206339,21.797610,21.635277,2.819609,1.007935,0.408729,0.162333,1.416664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23448612,269.008259,-35.997534,355.036417,-5.487381,1,9.69,0.787,1,4.604,0.04,...,23.311139,21.015516,20.240226,19.999518,19.899340,2.295623,0.775290,0.240707,0.100179,1.015998
23448613,269.007828,-35.997489,355.036283,-5.487056,4,10.01,0.902,1,4.423,0.66,...,22.645588,20.540339,19.878242,19.708969,19.660151,2.105249,0.662097,0.169273,0.048818,0.831370
23448614,269.008260,-35.997283,355.036637,-5.487257,1,9.89,0.407,1,4.837,0.04,...,27.921248,25.228468,24.124048,22.954961,22.455623,2.692780,1.104419,1.169088,0.499338,2.273507
23448615,269.007053,-35.997056,355.036347,-5.486296,2,10.05,0.420,1,4.853,-0.82,...,27.717221,25.217613,24.070581,23.540204,23.295803,2.499608,1.147032,0.530377,0.244401,1.677409


In [176]:
result = qc.query(sql=query)

In [177]:
trilegal = convert(result)

In [178]:
trilegal

,ra,dec,nest4096
0,29.503980,-0.495855,72977464
1,29.504495,-0.491694,72977464
2,29.513602,-0.494880,72977464
3,29.511452,-0.487658,72977464
4,29.512957,-0.491590,72977464
...,...,...,...
20293,30.481812,0.495479,72992955
20294,30.477584,0.498078,72992955
20295,30.482111,0.497589,72992955
20296,30.496184,0.494280,72992956


In [181]:
print(np.min(trilegal['nest4096']), np.max(trilegal['nest4096'])) 

72956637 72993314


In [159]:
rootname = 'priorsDir/BDBS/BDBSpatch1/BDBSpatch1-priorMap' 
mp.dumpPriorsFromPatch(BDBSpatch1, rootname, show2Dmap=False, verbose=False)

done with priorsDir/BDBS/BDBSpatch1/BDBSpatch1-priorMap


In [160]:
outfilename = 'priorsDir/BDBS/BDBSpatch1/BDBSpatch1-TRILEGALcatalog.txt' 
mp.dumpCatalogFromPatch(BDBSpatch1, outfilename)

dumping catalog...
made catalog: priorsDir/BDBS/BDBSpatch1/BDBSpatch1-TRILEGALcatalog.txt


## dump priors for multiple fields from SDSS Stripe 82

In [166]:
RAlist = [310, 320, 330, 340, 350, 0, 10, 20, 30, 40]
iPatch = 0 
for RA in RAlist:
    RAmin = RA
    RAmax = RA+10 
    querySDSSpatch = mp.makeQueryString(RAmin, RAmax, -1.5, 1.5)
    SDSSpatch = mp.retrievePatch(querySDSSpatch)
    rootname = 'priorsDir/Stripe82/SDSSpatchRA' + str(RAmin) + '-' + str(RAmax) + '-priorMap' 
    mp.dumpPriorsFromPatch(SDSSpatch, rootname, show2Dmap=False, verbose=False)
    print(rootname)
    iPatch += 1 
    print('--- done with patch', iPatch,' ---')

querying...
converting...
retrieved patch with 2991903  entries
done with priorsDir/Stripe82/SDSSpatchRA310-320-priorMap
priorsDir/Stripe82/SDSSpatchRA310-320-priorMap
--- done with patch 1  ---
querying...
converting...
retrieved patch with 1809831  entries
done with priorsDir/Stripe82/SDSSpatchRA320-330-priorMap
priorsDir/Stripe82/SDSSpatchRA320-330-priorMap
--- done with patch 2  ---
querying...
converting...
retrieved patch with 1236787  entries
done with priorsDir/Stripe82/SDSSpatchRA330-340-priorMap
priorsDir/Stripe82/SDSSpatchRA330-340-priorMap
--- done with patch 3  ---
querying...
converting...
retrieved patch with 939040  entries
done with priorsDir/Stripe82/SDSSpatchRA340-350-priorMap
priorsDir/Stripe82/SDSSpatchRA340-350-priorMap
--- done with patch 4  ---
querying...
converting...
retrieved patch with 777970  entries
done with priorsDir/Stripe82/SDSSpatchRA350-360-priorMap
priorsDir/Stripe82/SDSSpatchRA350-360-priorMap
--- done with patch 5  ---
querying...
converting...
r